In [1]:
import pandas as pd

# ファイルパス
train_file_path = 'dataset_ica_ml_train.csv'
test_file_path = 'dataset_ica_ml_test.csv'

# データの読み込み
train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

# データ概要
print("Train Data Information:")
train_data.info()

print("\nTest Data Information:")
test_data.info()

# データの先頭5行を表示
print("\nTrain Data Head:")
print(train_data.head())

print("\nTest Data Head:")
print(test_data.head())

# 各カラムの欠損値を確認
print("\nMissing Values in Train Data:")
print(train_data.isnull().sum())

print("\nMissing Values in Test Data:")
print(test_data.isnull().sum())

# 統計情報の表示
print("\nStatistical Summary of Train Data:")
print(train_data.describe())

print("\nStatistical Summary of Test Data:")
print(test_data.describe())

Train Data Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 32 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
 11  expenditure_1    10000 non-null  int64  
 12  expenditure_2    10000 non-null  int64  
 13  expenditure_3    10000 non-null  int64  
 14  expenditure_4    10000 non-null  int64  
 15  expenditure_5    10000 non-null  int64  
 16  expenditure_6    10000 non-null  in